In [ ]:
import pandas as pd
import numpy as np

dt = 1
f = np.array([[1, dt], [0, 1.1]])
g = np.array([[0.5], [1]])
h = np.identity(2)

df = pd.read_csv(r'Data_Sept_F180_Combined.csv')

output = pd.DataFrame(columns=['X', 'Y', 'H', 'Vx', 'Vy', 'Vh'])

for idx, row in df.iterrows():
    qx = np.diag([float(row['sde(m)'])**2, 0.014**2])
    qy = np.diag([float(row['sdn(m)'])**2, 0.014**2])
    qh = np.diag([float(row['sdu(m)'])**2, 0.014**2])

    if idx == 0:
        x_prediction = np.array([float(row['X (m)']), float(row['Ve(m/s)'])])
        px_prediction = np.array([[23.8925596, 0.000141826996], [0.000141826996, 0.000317134573]])
        y_prediction = np.array([float(row['Y (m)']), float(row['Vn (m/s)'])])
        py_prediction = np.array([[23.8925596, 0.000141826996], [0.000141826996, 0.000317134573]])
        h_prediction = np.array([float(row['height(m)']), float(row['Vu(m/s)'])])
        ph_prediction = np.array([[23.8925596, 0.000141826996], [0.000141826996, 0.000317134573]])

    else:
        x_correction_rhs = h @ x_prediction - np.array([[float(row['X (m)'])], [float(row['Ve(m/s)'])]])
        x_kalman_rhs = h @ px_prediction @ h.T + qx
        x_kalman = px_prediction @ h.T @ np.linalg.inv(x_kalman_rhs)
        x_correction = x_prediction - x_kalman @ x_correction_rhs

        y_correction_rhs = h @ y_prediction - np.array([[float(row['Y (m)'])], [float(row['Vn (m/s)'])]])
        y_kalman_rhs = h @ py_prediction @ h.T + qy
        y_kalman = py_prediction @ h.T @ np.linalg.inv(y_kalman_rhs)
        y_correction = y_prediction - y_kalman @ y_correction_rhs

        h_correction_rhs = h @ h_prediction - np.array([[float(row['height(m)'])], [float(row['Vu(m/s)'])]])
        h_kalman_rhs = h @ ph_prediction @ h.T + qh
        h_kalman = ph_prediction @ h.T @ np.linalg.inv(h_kalman_rhs)
        h_correction = h_prediction - h_kalman @ h_correction_rhs

        x_prediction = f @ x_correction 
        px_prediction = f @ px_correction @ f.T + qx
        y_prediction = f @ y_correction 
        py_prediction = f @ py_correction @ f.T + qy
        h_prediction = f @ h_correction 
        ph_prediction = f @ ph_correction @ f.T + qh
    
    rx = float(row['sde(m)'])**2
    ry = float(row['sdn(m)'])**2
    rh = float(row['sdu(m)'])**2

    x_kalman_rhs = h @ px_prediction @ h.T + qx
    y_kalman_rhs = h @ py_prediction @ h.T + qy
    h_kalman_rhs = h @ ph_prediction @ h.T + qh

    x_kalman = px_prediction @ h.T @ np.linalg.inv(x_kalman_rhs)
    y_kalman = py_prediction @ h.T @ np.linalg.inv(y_kalman_rhs)
    h_kalman = ph_prediction @ h.T @ np.linalg.inv(h_kalman_rhs)

    Yx = h @ np.array([[float(row['X (m)'])], [float(row['Ve(m/s)'])]]) + qx
    Yy = h @ np.array([[float(row['Y (m)'])], [float(row['Vn (m/s)'])]]) + qy
    Yh = h @ np.array([[float(row['height(m)'])], [float(row['Vu(m/s)'])]]) + qh

    x_correction_rhs = Yx - h @ x_prediction
    x_correction_rhs1 = x_kalman @ x_correction_rhs
    x_correction = x_prediction + x_correction_rhs1

    y_correction_rhs = Yy - h @ y_prediction
    y_correction_rhs1 = y_kalman @ y_correction_rhs
    y_correction = y_prediction + y_correction_rhs1

    h_correction_rhs = Yh - h @ h_prediction
    h_correction_rhs1 = h_kalman @ h_correction_rhs
    h_correction = h_prediction + h_correction_rhs1

    output = output.append({"X": x_correction[0, 0], "Y": y_correction[0,0], "H": h_correction[0, 0], 
                            "Vx": x_correction[1,0], "Vy": y_correction[1,0], "Vh": h_correction[1, 0]}, ignore_index=True)

    px_correction_lhs = np.identity(2) - x_kalman @ h
    px_correction = px_correction_lhs @ px_prediction

    py_correction_lhs = np.identity(2) - y_kalman @ h
    py_correction = py_correction_lhs @ py_prediction

    ph_correction_lhs = np.identity(2) - h_kalman @ h
    ph_correction = ph_correction_lhs @ ph_prediction

output.to_csv('DKF.csv')